In [1]:
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np

from ssd import SSD
from ssd_training import MultiboxLoss
from ssd_utils import Generator, create_bbox_with_priors

import pickle
from random import shuffle
import json

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [ ]:
def gen_fix(path, unseen_gen):
    images, filenames = next(unseen_gen)
    old_filenames = filenames
    for i, fname in enumerate(old_filenames):
        filenames[i] = path + "/" + fname
    return images, filenames

In [2]:
# some constants
NUM_CLASSES = 80
weight_load_num_classes = 80
target_size = (512, 512)
batch_size = 4

finetune = True
only_last_layers = True

weight_path = 'weights/weights_512x512.hdf5'
image_folder_path = 'my_training_images'
bbox_pickle_path = 'my_training_pickle.p'
num_freeze_layers = 33 if only_last_layers else 22
train_val_split = 0.9
with open('coco_labels.json', "r") as class_file:
    classes = json.load(class_file)

In [3]:
ssd = SSD(size=target_size, num_classes=weight_load_num_classes)
bbox_util = create_bbox_with_priors(ssd.model, NUM_CLASSES, target_size)

In [4]:
gt = pickle.load(open(bbox_pickle_path, 'rb'))
keys = gt.keys()
shuffle(keys)
num_train = int(round(train_val_split * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

In [5]:
gen = Generator(gt, bbox_util, batch_size, image_folder_path,
                train_keys, val_keys,
                target_size, do_crop=False)

In [ ]:
checkpoint = ModelCheckpoint('./checkpoints/weights.top_val-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [6]:
if finetune:
    ssd.load_weights_finetune(NUM_CLASSES, weight_path, False)
else:
    ssd.model.load_weights(weight_path, by_name=True)
model = ssd.model

In [7]:
base_lr = 0.0001
optim = keras.optimizers.Adam(lr=base_lr)
model.compile(optimizer=optim,
              loss=MultiboxLoss(NUM_CLASSES, neg_pos_ratio=3.0).compute_loss)

In [8]:
nb_epoch = 3
history = model.fit_generator(gen.generate(True), gen.train_batches,
                              nb_epoch, verbose=1,
                              callbacks=callbacks,
                              validation_data=gen.generate(False),
                              nb_val_samples=gen.val_batches,
                              nb_worker=1)